# ProcrastiGuard – Multi-Agent Study & Procrastination Assistant


Students often make good study plans but fail to follow them consistently due to procrastination. 
This project uses a multi-agent AI system to:
1. Create a realistic exam study plan
2. Track individual study sessions
3. Detect procrastination behavior
4. Suggest real-time interventions


## System Architecture

Planner Agent → Study Session Agent → Behavior Classifier Agent → Intervention Agent → Memory (Session History)

- Planner Agent: Creates long-term exam study plan
- Study Session Agent: Records what actually happened in a session
- Behavior Classifier Agent: Detects procrastination severity
- Intervention Agent: Suggests corrective actions
- Memory: Stores all session records


## Section 1: Environment Setup and Imports

In [1]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

✅ Gemini API key setup complete.


In [2]:
from google.adk.agents import Agent
from google.adk.models import Gemini
from google.adk.tools import google_search
from google.genai import types
from google.adk.runners import InMemoryRunner
from google.adk.tools import FunctionTool
from google.adk.agents import SequentialAgent

print("✅import completed.")

✅import completed.


In [3]:
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1, # Initial delay before first retry (in seconds)
    http_status_codes=[429, 500, 503, 504] # Retry on these HTTP errors
)

In [4]:
# In-memory storage for all completed study sessions
session_history = []
print("✅ session_history reset.")



✅ session_history reset.


## Section 2: Base Agent Definitions

In [5]:
planner_agent = Agent(
    name="planner_agent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    description="Helps students create a realistic study plan for exams.",
    instruction="""
You are a friendly and practical study planning assistant for students.

The user will provide:
- Exam date
- How many hours they can study per day
- A list of subjects with difficulty (1–5) or priority

Your job:
- Create a simple, realistic study plan.
- Allocate more time to higher-difficulty subjects.
- Avoid overloading a single day.
- Present the plan in a clear, structured format (for example: Day-wise or Subject-wise).
- Keep the tone encouraging and supportive.

If the user input is incomplete, politely ask what is missing.
""",
    tools=[],  # No external tools needed for now; it's pure planning logic
)

print("✅ Planner Agent defined.")


✅ Planner Agent defined.


In [6]:
planner_runner = InMemoryRunner(agent=planner_agent)

print("✅ Planner Runner created.")

✅ Planner Runner created.


In [7]:
response = await planner_runner.run_debug(
    """
Exam date: 2025-12-15
Daily study time: 4 hours
Subjects:
- Math (difficulty 5)
- Physics (difficulty 4)
- Chemistry (difficulty 3)
"""
)

for event in response:
    if hasattr(event, "output_text") and event.output_text:
        print("\nplanner_agent >", event.output_text)



 ### Created new session: debug_session_id

User > 
Exam date: 2025-12-15
Daily study time: 4 hours
Subjects:
- Math (difficulty 5)
- Physics (difficulty 4)
- Chemistry (difficulty 3)

planner_agent > Hi there! I can definitely help you create a study plan to get ready for your exams. Let's get this organized so you feel prepared and confident!

Here's a realistic study plan based on the information you've provided:

**Exam Date:** 2025-12-15
**Daily Study Time:** 4 hours
**Subjects & Difficulty:**
*   Math (difficulty 5)
*   Physics (difficulty 4)
*   Chemistry (difficulty 3)

To make sure we give more attention to the tougher subjects, here's a possible breakdown:

*   **Math:** 2 hours per day
*   **Physics:** 1.5 hours per day
*   **Chemistry:** 0.5 hours per day

This way, Math gets the most focus, followed by Physics, and then Chemistry. We'll aim to distribute this evenly across your study days.

***

### Your Study Plan

Since your exam is on **2025-12-15**, we have a good amo

In [8]:
session_agent = Agent(
    name="study_session_agent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    description="Guides the student through a single study session and summarizes their behavior.",
   instruction="""
You are an assistant that manages ONE study session for a student.

Your tasks:
1. Normally, you ask the user:
   - What subject they planned to study.
   - How many minutes they planned to study.
   - Whether they started on time or late.
   - Roughly how many minutes they actually studied.
   - If there was any delay or distraction, ask why.

2. BUT sometimes the user will give you all this information in one long message.
   - In that case, DO NOT ask any follow-up questions.
   - Just use the provided information to build the summary.

3. At the end, output a **clear structured summary** of the session in the following format:

SESSION SUMMARY:
- subject: <subject name>
- planned_minutes: <integer>
- actual_minutes: <integer>
- delay_minutes: <integer, 0 if none>
- reason_for_delay: <short text or 'none'>
- self_rating: <1-5, where 5 = very focused>

Rules:
- If you still need information, ask questions one by one, in a friendly and encouraging tone.
- If everything is already clear, skip questions and go straight to the SESSION SUMMARY.
- At the end, always output the SESSION SUMMARY block exactly in that format.
""",
    tools=[],  # no external tools yet; we keep it simple
)

print("✅ Study Session Agent defined.")


✅ Study Session Agent defined.


In [9]:
session_runner = InMemoryRunner(agent=session_agent)
print("✅ Session Runner created.")

✅ Session Runner created.


In [10]:
response = await session_runner.run_debug(
    "I want to start a study session."
)

for event in response:
    if hasattr(event, "output_text") and event.output_text:
        print("\nstudy_session_agent >", event.output_text)



 ### Created new session: debug_session_id

User > I want to start a study session.
study_session_agent > Great! What subject are you planning to study today? And for how many minutes?


In [11]:
def calculate_delay(planned_minutes: int, actual_minutes: int) -> dict:
    """
    Calculates delay between planned and actual study time.

    Args:
        planned_minutes: How many minutes were planned.
        actual_minutes: How many minutes were actually studied.

    Returns:
        A dict with:
        - delay_minutes: non-negative integer, 0 if no delay.
    """
    delay = planned_minutes - actual_minutes
    if delay < 0:
        delay = 0
    return {"delay_minutes": delay}


In [12]:
behavior_classifier_agent = Agent(
    name="behavior_classifier_agent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    description="Analyzes a study session summary and classifies procrastination behavior.",
    instruction="""
You are an analytical assistant that classifies student procrastination behavior
based on a single study session summary.

You will receive a SESSION SUMMARY in this format:

SESSION SUMMARY:
- subject: <subject name>
- planned_minutes: <integer>
- actual_minutes: <integer>
- delay_minutes: <integer>
- reason_for_delay: <short text or 'none'>
- self_rating: <1-5>

Your tasks:
1. Analyze the data and decide:
   - procrastination_type: one of
     [ "on_track", "mild_delay", "moderate_delay", "severe_procrastination" ]
   - severity: one of [ "low", "medium", "high" ]
2. Write a SHORT explanation of why you chose this classification.

Your output MUST be in this exact format:

DIAGNOSIS:
- procrastination_type: <one of the above>
- severity: <low/medium/high>
- explanation: <one short sentence>

Be concise and objective.
""",
    tools=[],  # no tools needed yet
)

print("✅ Behavior Classifier Agent defined.")


✅ Behavior Classifier Agent defined.


In [13]:
classifier_runner = InMemoryRunner(agent=behavior_classifier_agent)
print("✅ Classifier Runner created.")

✅ Classifier Runner created.


In [14]:
session_summary = """
SESSION SUMMARY:
- subject: Math
- planned_minutes: 60
- actual_minutes: 20
- delay_minutes: 30
- reason_for_delay: Scrolling on phone
- self_rating: 2
"""

response = await classifier_runner.run_debug(session_summary)

for event in response:
    if hasattr(event, "output_text") and event.output_text:
        print("\nbehavior_classifier_agent >", event.output_text)


 ### Created new session: debug_session_id

User > 
SESSION SUMMARY:
- subject: Math
- planned_minutes: 60
- actual_minutes: 20
- delay_minutes: 30
- reason_for_delay: Scrolling on phone
- self_rating: 2

behavior_classifier_agent > DIAGNOSIS:
- procrastination_type: moderate_delay
- severity: medium
- explanation: The actual study time was significantly less than planned, with a substantial delay attributed to phone usage, aligning with a medium severity level.


In [15]:
intervention_agent = Agent(
    name="intervention_agent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    description="Suggests practical interventions to reduce procrastination based on diagnosis.",
    instruction="""
You are a friendly but practical coach who helps students reduce procrastination.

You will receive a DIAGNOSIS in this format:

DIAGNOSIS:
- procrastination_type: <on_track | mild_delay | moderate_delay | severe_procrastination>
- severity: <low | medium | high>
- explanation: <one short sentence>

Your tasks:
1. Suggest 2–3 concrete, realistic actions the student can take in their NEXT study session.
2. Tailor the advice to the procrastination_type and severity.
3. Keep the tone encouraging but honest.

Your output MUST be in this format:

INTERVENTION PLAN:
- summary: <one short sentence>
- actions:
  1) <first action>
  2) <second action>
  3) <optional third action>

Do NOT add anything outside this structure.
""",
    tools=[],  # can add tools later if needed
)

print("✅ Intervention Agent defined.")


✅ Intervention Agent defined.


In [16]:
intervention_runner = InMemoryRunner(agent=intervention_agent)
print("✅ Intervention Runner created.")

✅ Intervention Runner created.


In [17]:
diagnosis_text = """
DIAGNOSIS:
- procrastination_type: moderate_delay
- severity: medium
- explanation: The actual study time was significantly less than planned, with a notable delay attributed to phone use, indicating moderate procrastination.
"""

response = await intervention_runner.run_debug(diagnosis_text)

for event in response:
    if hasattr(event, "output_text") and event.output_text:
        print("\nintervention_agent >", event.output_text)


 ### Created new session: debug_session_id

User > 
DIAGNOSIS:
- procrastination_type: moderate_delay
- severity: medium
- explanation: The actual study time was significantly less than planned, with a notable delay attributed to phone use, indicating moderate procrastination.

intervention_agent > INTERVENTION PLAN:
- summary: Let's get you back on track by structuring your study time and minimizing distractions.
- actions:
  1) Use a timer for focused study bursts (e.g., 25 minutes) followed by short, timed breaks (e.g., 5 minutes). During breaks, avoid checking your phone.
  2) Before you start, identify one specific, small task you want to accomplish in this session. This will make it easier to begin and feel a sense of achievement.
  3) Keep your phone in another room or turn off notifications during your study sessions to reduce the temptation to get sidetracked.


## Section 3: Agent Workflow Integration and Final Output

In [18]:
study_pipeline_agent = SequentialAgent(
    name="procrastiguard_planner_pipeline",
    description="Exam planning workflow: creates a realistic, difficulty-aware study plan.",
    sub_agents=[
        planner_agent,  # ✅ Only planner here
    ],
)

print("✅ Planner-only SequentialAgent defined.")

pipeline_runner = InMemoryRunner(agent=study_pipeline_agent)
print("✅ Pipeline runner created.")


✅ Planner-only SequentialAgent defined.
✅ Pipeline runner created.


In [19]:
#response = await pipeline_runner.run_debug(
 #   "Start a full study support flow for me."
#)
response = await pipeline_runner.run_debug(
    """
I want full support for my exam.

Exam date: 2025-12-15
Daily study time: 4 hours
Subjects:
- Math (difficulty 5)
- Physics (difficulty 4)
- Chemistry (difficulty 3)

Start the full study support flow for me.
"""
)

for event in response:
    if hasattr(event, "output_text") and event.output_text:
        print("\n[pipeline] >", event.output_text)



 ### Created new session: debug_session_id

User > 
I want full support for my exam.

Exam date: 2025-12-15
Daily study time: 4 hours
Subjects:
- Math (difficulty 5)
- Physics (difficulty 4)
- Chemistry (difficulty 3)

Start the full study support flow for me.

planner_agent > Hello there! I'd be absolutely delighted to help you get ready for your exams. It sounds like you've got a solid plan for your study time. Let's get this organized for you!

Based on your exam date of December 15th, 2025, and your daily study commitment of 4 hours, here's a possible study plan focusing on your subjects, with more time allocated to Math and Physics due to their higher difficulty:

### Your Study Plan

**Key:**
*   **Math:** Difficulty 5
*   **Physics:** Difficulty 4
*   **Chemistry:** Difficulty 3

We'll aim for a balanced approach, ensuring you cover all subjects without feeling overwhelmed on any single day. This plan assumes you're starting your studies soon.

---

**How to use this plan:**

*

In [20]:
async def run_single_session_with_analysis(session_description: str):

    print("====== STEP 1: Study Session Agent ======\n")

    session_record = {}

    # -------- STEP 1: SESSION AGENT --------
    events = await session_runner.run_debug(session_description)

    summary = None
    for e in events:
        # ✅ Only accept REAL strings, never None
        if hasattr(e, "output_text") and isinstance(e.output_text, str) and e.output_text.strip():
            summary = e.output_text.strip()
            break

    # ✅ Absolute safety fallback (won’t be used in your case)
    if summary is None:
        summary = (
            "SESSION SUMMARY:\n"
            "- subject: unknown\n"
            "- planned_minutes: 0\n"
            "- actual_minutes: 0\n"
            "- delay_minutes: 0\n"
            "- reason_for_delay: unknown\n"
            "- self_rating: 0"
        )
        print("⚠ Fallback SESSION SUMMARY used.")

    print("study_session_agent >", summary)
    session_record["session_summary"] = summary


    print("\n====== STEP 2: Behavior Classifier Agent ======\n")

    # -------- STEP 2: CLASSIFIER --------
    events = await classifier_runner.run_debug(summary)

    diagnosis = None
    for e in events:
        if hasattr(e, "output_text") and isinstance(e.output_text, str) and e.output_text.strip():
            diagnosis = e.output_text.strip()
            break

    if diagnosis is None:
        diagnosis = (
            "DIAGNOSIS:\n"
            "- procrastination_type: on_track\n"
            "- severity: low\n"
            "- explanation: No classifier output."
        )
        print("⚠ Fallback DIAGNOSIS used.")

    print("behavior_classifier_agent >", diagnosis)
    session_record["diagnosis"] = diagnosis


    print("\n====== STEP 3: Intervention Agent ======\n")

    # -------- STEP 3: INTERVENTION --------
    events = await intervention_runner.run_debug(diagnosis)

    intervention = None
    for e in events:
        if hasattr(e, "output_text") and isinstance(e.output_text, str) and e.output_text.strip():
            intervention = e.output_text.strip()
            break

    if intervention is None:
        intervention = (
            "INTERVENTION PLAN:\n"
            "- summary: No intervention generated.\n"
            "- actions:\n"
            "  1) Try again."
        )
        print("⚠ Fallback INTERVENTION used.")

    print("intervention_agent >", intervention)
    session_record["intervention"] = intervention


    # -------- SAVE --------
    session_history.append(session_record)
    return session_record



async def run_single_session_with_analysis(session_description: str):

    print("====== STEP 1: Study Session Agent ======\n")

    session_record = {}

    # ---------- STEP 1: SESSION AGENT ----------
    events = await session_runner.run_debug(session_description)

    summary = None
    for e in events:
        if hasattr(e, "output_text") and isinstance(e.output_text, str):
            summary = e.output_text
            break   # ✅ FIRST real output only

    # ✅ Absolute safety check (no fallback overwrite)
    if summary is None:
        raise RuntimeError("Session agent returned no usable output.")

    print("study_session_agent >", summary)
    session_record["session_summary"] = summary


    print("\n====== STEP 2: Behavior Classifier Agent ======\n")

    events = await classifier_runner.run_debug(summary)

    diagnosis = None
    for e in events:
        if hasattr(e, "output_text") and isinstance(e.output_text, str):
            diagnosis = e.output_text
            break

    if diagnosis is None:
        raise RuntimeError("Behavior classifier returned no usable output.")

    print("behavior_classifier_agent >", diagnosis)
    session_record["diagnosis"] = diagnosis


    print("\n====== STEP 3: Intervention Agent ======\n")

    events = await intervention_runner.run_debug(diagnosis)

    intervention = None
    for e in events:
        if hasattr(e, "output_text") and isinstance(e.output_text, str):
            intervention = e.output_text
            break

    if intervention is None:
        raise RuntimeError("Intervention agent returned no usable output.")

    print("intervention_agent >", intervention)
    session_record["intervention"] = intervention


    # ---------- SAVE ----------
    session_history.append(session_record)
    return session_record




   


In [21]:
def show_session_history():
    if not session_history:
        print("No sessions recorded yet.")
        return

    for i, rec in enumerate(session_history, start=1):
        print(f"\n================= SESSION {i} =================\n")

        print("---- RAW RECORD ----")
        print(repr(rec))  # show the actual dict, so we see keys & values
        print("\n--------------------\n")

        print("---- SESSION SUMMARY ----")
        print(rec.get("session_summary", "[missing key]"))
        print("\n---- DIAGNOSIS ----")
        print(rec.get("diagnosis", "[missing key]"))
        print("\n---- INTERVENTION ----")
        print(rec.get("intervention", "[missing key]"))
        print("\n==============================================\n")


In [22]:
example_session = """
You already have all the information you need. Don't ask me any questions.
Just output the SESSION SUMMARY in the required format.

Here is what happened in my study session:

- subject: Math
- I planned to study for 60 minutes.
- I actually studied for 20 minutes.
- I started about 30 minutes late.
- The main reason for delay: I was scrolling on my phone and checking social media.
- On a scale of 1-5, I would rate my focus as 2.
"""
session_history.clear()

await run_single_session_with_analysis(example_session)

show_session_history()


====== STEP 1: Study Session Agent ======


 ### Continue session: debug_session_id

User > 
You already have all the information you need. Don't ask me any questions.
Just output the SESSION SUMMARY in the required format.

Here is what happened in my study session:

- subject: Math
- I planned to study for 60 minutes.
- I actually studied for 20 minutes.
- I started about 30 minutes late.
- The main reason for delay: I was scrolling on my phone and checking social media.
- On a scale of 1-5, I would rate my focus as 2.

study_session_agent > SESSION SUMMARY:
- subject: Math
- planned_minutes: 60
- actual_minutes: 20
- delay_minutes: 30
- reason_for_delay: Scrolling on phone and checking social media
- self_rating: 2
⚠ Fallback SESSION SUMMARY used.
study_session_agent > SESSION SUMMARY:
- subject: unknown
- planned_minutes: 0
- actual_minutes: 0
- delay_minutes: 0
- reason_for_delay: unknown
- self_rating: 0

====== STEP 2: Behavior Classifier Agent ======


 ### Continue session: de

## How to Run the Project

1. Add your Gemini API key to Kaggle Secrets as `GOOGLE_API_KEY`
2. Run all cells from top to bottom
3. The example study session will automatically run
4. Session history will be printed at the end
